# 基站覆盖问题

## 目标和前提条件

想学习如何配置基站网络以为尽可能多的人提供信号覆盖吗？在这个示例中，您将学习如何解决这个简单的覆盖问题。我们将向您展示如何构建该问题的混合整数规划(MIP)模型，如何在Gurobi Python API中实现这个模型，以及如何使用Gurobi优化器找到最优解。

这个建模示例属于入门级别，我们假设您了解Python，并且对构建数学优化模型有一定了解。

**下载代码库：** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 背景介绍

在过去十年中，智能手机彻底改变了我们的生活方式，其影响远超通信领域。除了打电话、发短信和收发电子邮件之外，全球现有超过20亿人使用这些设备进行导航、预约出租车、比较产品评价和价格、关注新闻、观看电影、听音乐、玩视频游戏、拍照、参与社交媒体以及众多其他应用。

蜂窝网络是一个由手持智能手机组成的网络，每部手机通过基站（基站塔）的本地天线以无线电波与电话网络进行通信。一个重要的问题是如何布置基站塔，以便为最多数量的人提供信号覆盖。

## 问题描述

某电信公司需要建设一系列基站塔，为特定城市的居民提供信号覆盖。已经确定了一些可能的基站建设位置。基站有固定的覆盖范围，而且由于预算限制，只能建设有限数量的基站。在这些限制条件下，公司希望为尽可能多的人口提供覆盖。为简化问题，公司将需要覆盖的区域划分为若干个区域，每个区域都有已知的人口数量。目标是选择在哪些潜在位置建设基站塔，以便为尽可能多的人提供覆盖。

基站覆盖问题是最大覆盖位置问题[1]的一个实例。它也与集合覆盖问题相关。集合覆盖问题在许多不同领域都有应用，航空业有非常重要的应用。例如，机组排班和飞机尾号分配问题[2]。

## 解决方案

数学规划是一种声明式方法，建模者通过制定数学优化模型来捕获复杂决策问题的关键方面。Gurobi优化器使用最先进的数学和计算机科学来解决这些模型。

数学优化模型有五个组成部分，即：

* 集合和索引
* 参数
* 决策变量
* 目标函数
* 约束条件

现在我们将为基站覆盖问题提出一个混合整数规划(MIP)模型。

## 模型公式

### 集合和索引

$i \in T$: 建设基站的潜在位置的索引和集合。

$j \in R$: 区域的索引和集合。

$G(T,R,E)$: 定义在潜在基站位置集合$T$、我们希望覆盖的区域集合$R$上的二分图，$E$是边的集合，如果区域$j \in R$可以被位置$i \in T$上的基站覆盖，则有边$(i,j) \in E$。

### 参数

$c_{i} \in \mathbb{R}^+$: 在位置$i$建设基站的成本。

$p_{j} \in \mathbb{N}$: 区域$j$的人口。

### 决策变量

$covered_{j} \in \{0, 1 \}$: 如果区域$j$被覆盖，则该变量等于1；否则等于0。

$build_{i} \in \{0, 1 \}$: 如果建设基站$i$，则该变量等于1；否则等于0。

### 目标函数

- **覆盖人口**。我们寻求最大化基站覆盖的总人口。

\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### 约束条件

- **覆盖**。对于每个区域$j \in R$，确保至少选择一个覆盖该区域的基站。

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **预算**。我们需要确保建设基站的总成本不超过分配的预算。

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## Python 实现

这个示例考虑了6个基站和9个区域的二分图。下表说明了每个基站位置（行）覆盖的区域（列）。

| <i></i> | 区域 0 | 区域 1 | 区域 2 | 区域 3 | 区域 4 | 区域 5 | 区域 6 | 区域 7 | 区域 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| 基站 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| 基站 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| 基站 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| 基站 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| 基站 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| 基站 5 | - | - | - | 1 | 1 | - | - | - |  1 |

每个区域的人口如下表所示。

| <i></i> | 区域 0 | 区域 1 | 区域 2 | 区域 3 | 区域 4 | 区域 5 | 区域 6 | 区域 7 | 区域 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 人口 | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

在每个位置建设基站的成本如下表所示。

| <i></i> | 成本（百万美元） |
| --- | --- |
| 基站 0 | 4.2 |
| 基站 1 | 6.1 |
| 基站 2 | 5.2 |
| 基站 3 | 5.5 |
| 基站 4 | 4.8 |
| 基站 5 | 9.2 | 

分配的预算为$\$20,000,000$。

现在我们导入Gurobi Python模块。然后，我们用给定的数据初始化数据结构。


In [ ]:
# %pip install gurobipy

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

# 参数
budget = 20
regions, population = gp.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

sites, coverage, cost = gp.multidict({
    0: [{0,1,5}, 4.2],
    1: [{0,7,8}, 6.1],
    2: [{2,3,4,6}, 5.2],
    3: [{2,5,6}, 5.5],
    4: [{0,2,6,7,8}, 4.8],
    5: [{3,4,8}, 9.2]
})


### 模型部署

现在我们确定基站覆盖问题的模型，定义决策变量、约束条件和目标函数。接下来，我们开始优化过程，Gurobi找到在给定预算下最大化人口覆盖的基站建设计划。

In [7]:
# MIP模型公式
m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize() 

Set parameter LicenseID to value 2601452
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xf0a21eec
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 10 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 18 available processors)

Solution count 2: 7051 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.051000000000e+03, best bound 7.051000000000e+03, gap 0.0000%

## 分析
优化模型的结果显示，在$\$20,000,000$预算下，最大可覆盖人口为7,051人。让我们看看实现该最优结果的解决方案。

### 基站建设计划

该计划确定在哪些位置建设基站。

In [8]:
# 显示决策变量的最优值

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower}.")


 Build a cell tower at location Tower 0.

 Build a cell tower at location Tower 4.

 Build a cell tower at location Tower 5.


### 需求满足指标

- **覆盖率**：基站覆盖的人口百分比。

In [9]:
# 计算基站覆盖的人口百分比。

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")


 The population coverage associated to the cell towers build plan is: 100.0 %


### 资源利用指标

- **预算消耗**：建设基站所消耗的预算百分比。

In [10]:
# 计算建设基站所消耗的预算百分比。

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The percentage of budget consumed associated to the cell towers build plan is: 91.0 %


## 结论

在这个示例中，我们解决了建设基站以为尽可能多的人提供信号覆盖的问题，同时满足预算约束。我们学习了如何将问题公式化为MIP模型。还学习了如何实现MIP模型公式并使用Gurobi Python API求解。

## 参考文献

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/

Copyright © 2020 Gurobi Optimization, LLC